In [ ]:
import pandas as pd
import os
import random
import json 
from faker import Faker
from os.path import join as path_join
from datetime import datetime as dt

Faker.seed(40028922)
faker_br = Faker('pt_BR')
faker_en = Faker('en_IN')
faker_jp = Faker('ja_JP')


CHUNKSIZE = 50000

In [4]:
def create_user():
    user = {}
    user['name'] = faker_br.name()
    user['email'] = faker_br.email()
    user['address'] = ' - '.join(faker_br.address().split('\n'))
    user['phone_number'] = faker_br.phone_number()
    return user

## Livros

In [5]:
df = pd.read_csv("livros/dataset.csv", nrows=10000)
df.head()

,authors,bestsellers-rank,categories,description,dimension-x,dimension-y,dimension-z,edition,edition-statement,for-ages,...,isbn10,isbn13,lang,publication-date,publication-place,rating-avg,rating-count,title,url,weight
0,[1],49848.0,"[214, 220, 237, 2646, 2647, 2659, 2660, 2679]",SOLDIER FIVE is an elite soldier's explosive m...,129.0,198.0,20.0,NaN,NaN,NaN,...,184018907X,9781840189070,en,2004-10-14 00:00:00,NaN,4.03,292.0,Soldier Five : The Real Truth About The Bravo ...,/Soldier-Five-Mike-Coburn/9781840189070,224.00
1,"[2, 3]",115215.0,"[235, 3386]",John Moran and Carl Williams were the two bigg...,127.0,203.2,25.4,NaN,NaN,NaN,...,184454737X,9781844547371,en,2009-03-13 00:00:00,NaN,3.60,335.0,Underbelly : The Gangland War,/Underbelly-Andrew-Rule/9781844547371,285.76
2,[4],11732.0,"[358, 2630, 360, 2632]",Sir Phillip knew that Eloise Bridgerton was a ...,150.0,224.0,28.0,New edition,NaN,NaN,...,8416327866,9788416327867,es,2020-04-30 00:00:00,NaN,3.88,37211.0,"A Sir Phillip, Con Amor",/Sir-Phillip-Con-Amor-Julia-Quinn/9788416327867,386.00
3,"[5, 6, 7, 8]",114379.0,"[377, 2978, 2980]",The Third Book of General Ignorance gathers t...,153.0,234.0,24.0,NaN,Export - Airside ed,NaN,...,571308996,9780571308996,en,2015-10-01 00:00:00,NaN,4.17,384.0,QI: The Third Book of General Ignorance,/QI-Third-Book-General-Ignorance-John-Lloyd/97...,436.00
4,[9],98413.0,"[2813, 2980]",The Try Guys deliver their first book-an inspi...,191.0,240.0,29.0,NaN,NaN,NaN,...,8352518,9780008352516,en,2019-06-18 00:00:00,NaN,3.90,5095.0,The Hidden Power of F*cking Up,/Hidden-Power-F-cking-Up-Try-Guys/9780008352516,980.00


In [6]:
df_final = df[['id', 'title', 'authors', 'categories', 'imprint', 'isbn13', 'lang', 'publication-date']]
df_final['pages'] = df['weight'].fillna(0.0).astype(int)
df_final['publisher'] = df['imprint']
df_final.head()

/tmp/ipykernel_7363/675217334.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['pages'] = df['weight'].fillna(0.0).astype(int)
/tmp/ipykernel_7363/675217334.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['publisher'] = df['imprint']


,id,title,authors,categories,imprint,isbn13,lang,publication-date,pages,publisher
0,9781840189070,Soldier Five : The Real Truth About The Bravo ...,[1],"[214, 220, 237, 2646, 2647, 2659, 2660, 2679]",Mainstream Publishing,9781840189070,en,2004-10-14 00:00:00,224,Mainstream Publishing
1,9781844547371,Underbelly : The Gangland War,"[2, 3]","[235, 3386]",NaN,9781844547371,en,2009-03-13 00:00:00,285,NaN
2,9788416327867,"A Sir Phillip, Con Amor",[4],"[358, 2630, 360, 2632]",NaN,9788416327867,es,2020-04-30 00:00:00,386,NaN
3,9780571308996,QI: The Third Book of General Ignorance,"[5, 6, 7, 8]","[377, 2978, 2980]",NaN,9780571308996,en,2015-10-01 00:00:00,436,NaN
4,9780008352516,The Hidden Power of F*cking Up,[9],"[2813, 2980]",HarperCollins Publishers Ltd,9780008352516,en,2019-06-18 00:00:00,980,HarperCollins Publishers Ltd


In [7]:
livros = 'livros/treated/data.csv'
os.remove('livros/treated/data.csv')

for idx, df in enumerate(pd.read_csv("livros/dataset.csv", iterator=True, chunksize=CHUNKSIZE)):
    df_final = df[['id', 'title', 'authors', 'categories', 'imprint', 'isbn13', 'lang', 'publication-date']]
    df_final['publication-date'] = pd.to_datetime(df['publication-date'], format='%Y-%m-%d', errors='coerce').dt.strftime('%Y-%m-%d')
    df_final['pages'] = df['weight'].fillna(0.0).astype(int)
    df_final['publisher'] = df['imprint']

    df_final.to_csv(livros, mode='a', header=True if idx == 0 else False, index=False)

/tmp/ipykernel_7363/4075291343.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['publication-date'] = pd.to_datetime(df['publication-date'], format='%Y-%m-%d', errors='coerce').dt.strftime('%Y-%m-%d')
/tmp/ipykernel_7363/4075291343.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['pages'] = df['weight'].fillna(0.0).astype(int)
/tmp/ipykernel_7363/4075291343.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

## Revistas      

In [8]:
COMIC_PUBLISHER = [
    "Image Comics","IDW Publishing","Dynamite Entertainment","BOOM! Studios","Archie Comics","Oni Press","Vault Comics","AfterShock Comics","Valiant Comics","AWA Studios","Skybound Entertainment","Z2 Comics","Ahoy Comics","Black Mask Studios","Scout Comics",
    "Mad Cave Studios","Humanoids Publishing","Titan Comics (USA division)","Fantagraphics Books","Drawn & Quarterly (USA distribution)","Top Shelf Productions","NBM Publishing","First Second Books","Papercutz","Seven Seas Entertainment","Tokyopop (USA division)","Udon Entertainment"
]


MANGA_PUBLISHER = [
    "Shueisha", "Kodansha", "Shogakukan", "Kadokawa Shoten", "Square Enix", "Akita Shoten", "Futabasha", "Hakusensha", "Ichijinsha", "Media Factory", "Enterbrain",
    "Houbunsha","ASCII Media Works","Shinchōsha","Takeshobo","Shonen Gahosha","Bunkasha","Leed Publishing","Tokuma Shoten","Core Magazine","Ohzora Publishing","Wani Books","Mag Garden","Shobunkan","Kaiōsha"
]

In [9]:

json_data = []
counter = 0
with open('revistas/goodreads_books_comics_graphic.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        data = json.loads(line)

        pub_date = ''        
        if data['publication_year'] and data['publication_month'] and data['publication_day']:
            pub_date = f"{data['publication_year']}-{data['publication_month']}-{data['publication_day']}"
        else:
            pubdate = faker_br.date_between(start_date='-30y', end_date='today').strftime('%Y-%m-%d')

        treated = { }
        treated['titulo'] = data['title'] 
        treated['autores'] = faker_en.name() 
        treated['periodicidade'] = random.choice(['Diário', 'Semanal', 'Quinzenal', 'Mensal', 'Bimestral'])
        treated['data_publicacao'] = pub_date
        treated['editora'] = data['publisher'] if data['publisher'] else random.choice(COMIC_PUBLISHER)
        treated['ISSN'] = data['isbn'] if data['isbn'] else faker_br.isbn13()

        json_data.append(treated)
        if counter % 1000 == 0:
            df = df
            pd.DataFrame(json_data)[['titulo', 'autores', 'periodicidade', 'data_publicacao', 'editora', 'ISSN']]\
                .to_csv('revistas/treated/goodreads_books_comics_graphic.csv', mode='a', header=True if counter == 0 else False, index=False)
            json_data = []
            print(f"Processed {counter} records")
        counter += 1

if json_data:
    pd.DataFrame(json_data)[['titulo', 'autores', 'periodicidade', 'data_publicacao', 'editora', 'ISSN']]\
        .to_csv('revistas/goodreads_books_comics_graphic.csv', mode='a', header=False, index=False)



Processed 0 records
Processed 1000 records
Processed 2000 records
Processed 3000 records
Processed 4000 records
Processed 5000 records
Processed 6000 records
Processed 7000 records
Processed 8000 records
Processed 9000 records
Processed 10000 records
Processed 11000 records
Processed 12000 records
Processed 13000 records
Processed 14000 records
Processed 15000 records
Processed 16000 records
Processed 17000 records
Processed 18000 records
Processed 19000 records
Processed 20000 records
Processed 21000 records
Processed 22000 records
Processed 23000 records
Processed 24000 records
Processed 25000 records
Processed 26000 records
Processed 27000 records
Processed 28000 records
Processed 29000 records
Processed 30000 records
Processed 31000 records
Processed 32000 records
Processed 33000 records
Processed 34000 records
Processed 35000 records
Processed 36000 records
Processed 37000 records
Processed 38000 records
Processed 39000 records
Processed 40000 records
Processed 41000 records
Proce

In [10]:
# index,Job,Title,Description,Writer,Artist,Colorist,Cover Artist,Publication Date,Format,Price,UPC,Editor,Letterer,Penciller,Inker

# titulo,autor,periodicidade,data_publicacao,editora,issn

author_cols = ['Writer','Artist','Colorist','Cover Artist']

df = pd.read_csv("revistas/DH_archive.csv", dtype=str, sep=",")

author_series, periodicidade = [], []
for _, row in df.iterrows():
    # print(row)
    authors = ' | '.join({ ' | '.join(row[col].split(',')) for col in author_cols if row[col] and row[col] != "Na" })
    author_series.append(authors)
    periodicidade.append(random.choice(['Diário', 'Semanal', 'Quinzenal', 'Mensal']))

# Transform date like "March 25, 2020" to "2020-03-25"
df['Publication Date'] = pd.to_datetime(df['Publication Date'], format='%B %d, %Y').dt.strftime('%Y-%m-%d')
df['ISSN'] = df['UPC'].str.replace(" ", "")


df = df[['Title', 'Publication Date', 'ISSN']].rename(columns={'Title': 'titulo', 'Publication Date': 'data_publicacao'})
df['editora'] = 'Dark Horse Comics'
df['autores'] = pd.Series(author_series)
df['periodicidade'] = pd.Series(periodicidade)

df = df[['titulo', 'autores', 'periodicidade', 'data_publicacao', 'editora', 'ISSN']]
df.to_csv("revistas/treated/DH_archive.csv", header=True, index=False)
df.head()

,titulo,autores,periodicidade,data_publicacao,editora,ISSN
0,Frankenstein Undone #3,Mike Mignola | Scott Allie | Ben Stenbeck | B...,Diário,2020-03-25,Dark Horse Comics,76156800538700311
1,Frankenstein Undone #3 (Steve Purcell Variant ...,Mike Mignola | Scott Allie | Steve Purcell | ...,Diário,2020-03-25,Dark Horse Comics,76156800538700321
2,Hidden Society #2,Rafael Albuquerque | Rafael Scavone | Marcelo ...,Mensal,2020-03-25,Dark Horse Comics,76156800626100211
3,Hidden Society #2 (Greg Tocchini Variant Cover),Rafael Albuquerque | Rafael Scavone | Greg Toc...,Semanal,2020-03-25,Dark Horse Comics,76156800626100221
4,Tomorrow #2,Peter Milligan | James Devlin | Jesus Hervas,Mensal,2020-03-25,Dark Horse Comics,76156800627800211


In [11]:
# titulo,autor,periodicidade,data_publicacao,editora,issn

df = pd.read_csv("revistas/data.csv", dtype=str, sep=",")

days, months = list(range(31))[1:], range(13)[1:]
pubdates = []
for row in df.itertuples():
    month, day = random.choice(months), random.choice(days)

    if month == 2 and day > 28: day = 7
    pubdates.append(dt(int(row[4]), month, day).strftime('%Y-%m-%d'))

df = df[["title"]].rename(columns={ "title": "titulo" })
df['autores'] = pd.Series([faker_jp.romanized_name() for _ in range(df.shape[0])])
df['ISSN'] = pd.Series([faker_en.isbn13() for _ in range(df.shape[0])])
df['editora'] = pd.Series([random.choice(MANGA_PUBLISHER) for _ in range(df.shape[0])])
df['periodicidade'] = pd.Series([random.choice(['Diário', 'Semanal', 'Quinzenal', 'Mensal']) for _ in range(df.shape[0])])
df['data_publicacao'] = pd.Series(pubdates)

df = df[['titulo', 'autores', 'periodicidade', 'data_publicacao', 'editora', 'ISSN']]
df.to_csv("revistas/treated/mangas.csv", header=True, index=False)
df.head(10)

,titulo,autores,periodicidade,data_publicacao,editora,ISSN
0,Salad Days (Tang LiuZang) - Part 2,Maaya Miura,Diário,2021-08-27,Wani Books,978-0-8085-6825-4
1,The Master of Diabolism,Jun Kondo,Quinzenal,2017-05-11,Shueisha,978-0-9940210-3-8
2,JoJo's Bizarre Adventure Part 7: Steel Ball Run,Jun Yamazaki,Quinzenal,2004-11-15,Takeshobo,978-1-4282-3444-4
3,A Sign of Affection,Kaori Abe,Semanal,2019-04-30,Leed Publishing,978-0-568-47043-9
4,Moriarty the Patriot,Yasuhiro Shimizu,Diário,2016-11-11,Futabasha,978-0-392-29371-6
5,Classroom of the Elite (Light Novel),Kana Saito,Diário,2015-12-03,Tokuma Shoten,978-1-109-98091-2
6,Who Made Me a Princess,Sayuri Suzuki,Semanal,2017-03-28,Core Magazine,978-0-920495-14-8
7,Demon Slayer: Kimetsu no Yaiba,Shota Tanaka,Semanal,2016-01-14,Tokuma Shoten,978-1-57388-590-4
8,Omniscient Reader (Novel),Hiroshi Suzuki,Diário,2018-11-18,Wani Books,978-1-135-17460-6
9,Omniscient Reader,Sotaro Ishikawa,Semanal,2020-07-04,Bunkasha,978-1-67331-983-5


In [12]:
# id,Name,Writer,Likes,Genre,Rating,Subscribers,Summary,Update,Reading Link

# titulo,autor,periodicidade,data_publicacao,editora,issn

df = pd.read_csv("revistas/Webtoon Dataset.csv", dtype=str, sep=",")

days, months = list(range(31))[1:], range(13)[1:]
pubdates = []
for row in df.itertuples():
    month, day = random.choice(months), random.choice(days)

    if month == 2 and day > 28: day = 13
    pubdates.append(dt(random.randint(1998, 2024), month, day).strftime('%Y-%m-%d'))


df = df[['Name','Writer']].rename(columns={'Name': 'titulo', 'Writer': 'autores'})

df['ISSN'] = pd.Series([faker_en.isbn13() for _ in range(df.shape[0])])

df['periodicidade'] = pd.Series([random.choice(['Diário', 'Semanal', 'Quinzenal', 'Mensal']) for _ in range(df.shape[0])])
df['editora'] = 'Webtoon'
df['data_publicacao'] = pd.Series(pubdates)

df = df[['titulo', 'autores', 'periodicidade', 'data_publicacao', 'editora', 'ISSN']]
df.to_csv("revistas/treated/Webtoon Dataset.csv", header=True, index=False)
df.head(10)


,titulo,autores,periodicidade,data_publicacao,editora,ISSN
0,Let's Play,Leeanne M. Krecic (Mongie),Quinzenal,2018-06-26,Webtoon,978-1-4400-0969-3
1,True Beauty,Yaongyi,Diário,2007-06-20,Webtoon,978-0-210-94585-8
2,Midnight Poppy Land,Lilydusk,Mensal,2015-08-28,Webtoon,978-1-368-20358-6
3,Age Matters,Enjelicious,Mensal,2012-11-21,Webtoon,978-0-7300-0633-6
4,Unholy Blood,Lina Im / Jeonghyeon Kim,Quinzenal,2013-02-24,Webtoon,978-0-8477-2436-9
5,LUMINE,Emma Krogell,Semanal,2001-02-25,Webtoon,978-0-350-58137-4
6,Jackson's Diary,Paola Batalla,Mensal,2006-10-05,Webtoon,978-1-350-43739-5
7,Teenage Mercenary,YC / Rakyeon,Diário,2014-11-25,Webtoon,978-1-65076-052-0
8,Nice To Meet You,Wishroomness,Quinzenal,2011-02-06,Webtoon,978-0-583-02923-0
9,I Love Yoo,Quimchee,Quinzenal,2023-02-13,Webtoon,978-0-523-39164-9


In [13]:
# id,Name,Writer,Likes,Genre,Rating,Subscribers,Summary,Update,Reading Link

# titulo,autor,periodicidade,data_publicacao,editora,issn

df = pd.read_csv("revistas/Webtoon Dataset.csv", dtype=str, sep=",")

days, months = list(range(31))[1:], range(13)[1:]
pubdates = []
for row in df.itertuples():
    month, day = random.choice(months), random.choice(days)

    if month == 2 and day > 28: day = 13
    pubdates.append(dt(random.randint(1998, 2024), month, day).strftime('%Y-%m-%d'))


df = df[['Name','Writer']].rename(columns={'Name': 'titulo', 'Writer': 'autores'})

df['ISSN'] = pd.Series([faker_en.isbn13() for _ in range(df.shape[0])])

df['periodicidade'] = pd.Series([random.choice(['Diário', 'Semanal', 'Quinzenal', 'Mensal']) for _ in range(df.shape[0])])
df['editora'] = 'Webtoon'
df['data_publicacao'] = pd.Series(pubdates)

df = df[['titulo', 'autores', 'periodicidade', 'data_publicacao', 'editora', 'ISSN']]
df.to_csv("revistas/treated/Webtoon Dataset.csv", header=True, index=False)
df.head(10)


,titulo,autores,periodicidade,data_publicacao,editora,ISSN
0,Let's Play,Leeanne M. Krecic (Mongie),Semanal,2006-03-19,Webtoon,978-0-493-94093-9
1,True Beauty,Yaongyi,Mensal,2002-05-26,Webtoon,978-1-4708-0772-6
2,Midnight Poppy Land,Lilydusk,Mensal,2014-05-01,Webtoon,978-0-08-490204-9
3,Age Matters,Enjelicious,Mensal,2020-08-23,Webtoon,978-1-175-69517-8
4,Unholy Blood,Lina Im / Jeonghyeon Kim,Diário,2018-09-30,Webtoon,978-1-00-220139-8
5,LUMINE,Emma Krogell,Semanal,2018-03-18,Webtoon,978-0-04-017789-4
6,Jackson's Diary,Paola Batalla,Semanal,2017-11-06,Webtoon,978-1-4888-6263-2
7,Teenage Mercenary,YC / Rakyeon,Semanal,2016-03-18,Webtoon,978-0-07-663183-4
8,Nice To Meet You,Wishroomness,Semanal,2016-03-12,Webtoon,978-1-4074-8145-6
9,I Love Yoo,Quimchee,Mensal,2004-01-04,Webtoon,978-0-332-67152-9


In [14]:
# index,Job,Title,Description,Writer,Artist,Colorist,Cover Artist,Publication Date,Format,Price,UPC,Editor,Letterer,Penciller,Inker

# titulo,autor,periodicidade,data_publicacao,editora,issn

author_cols = ['Writers','Pencilers','Inkers','Cover_Artists']

df = pd.read_csv("revistas/Complete_DC_Comic_Books.csv", dtype=str, sep=",")

pubdates = []
author_series, periodicidade = [], []
for _, row in df.iterrows():
    # print(row)
    authors = ' | '.join({ ' | '.join(str(row[col]).split(',')) for col in author_cols if row[col] })
    author_series.append(authors)
    periodicidade.append(random.choice(['Diário', 'Semanal', 'Quinzenal', 'Mensal']))

    year = str(row['Release_Date']).split(',')[-1].strip()
    year = year if year.isdecimal() else 2002
    pubdates.append(f"{year}-01-01")

df = df[['Issue_Name']].rename(columns={'Issue_Name': 'titulo'})
df['editora'] = 'DC Comics'
df['autores'] = pd.Series(author_series)
df['periodicidade'] = pd.Series(periodicidade)
df['ISSN'] = pd.Series([faker_en.isbn13() for _ in range(df.shape[0])])
df['data_publicacao'] = pd.Series(pubdates)

df = df[['titulo', 'autores', 'periodicidade', 'data_publicacao', 'editora', 'ISSN']]
df.to_csv("revistas/treated/Complete_DC_Comic_Books.csv", header=True, index=False)
df.head()

,titulo,autores,periodicidade,data_publicacao,editora,ISSN
0,Freddy vs. Jason vs. Ash #1,James Anthony Kuhoric | Jason Craig | nan | J....,Semanal,2008-01-01,DC Comics,978-0-472-65266-2
1,Freddy vs. Jason vs. Ash #2,James Kuhoric | Eric Powell | Jason Craig | nan,Quinzenal,2002-01-01,DC Comics,978-1-07-302714-9
2,Freddy vs. Jason vs. Ash #3,Eric Powell | Jason Craig | nan | James Anthon...,Quinzenal,2008-01-01,DC Comics,978-1-4969-2574-9
3,Freddy vs. Jason vs. Ash #4,Eric Powell | Jeff Kat | James Anthony Kuhori...,Semanal,2008-01-01,DC Comics,978-0-923381-31-8
4,Freddy vs. Jason vs. Ash #5,James Anthony Kuhoric | Jason Craig | Richard ...,Diário,2008-01-01,DC Comics,978-0-634-18723-0


In [15]:
# index,Job,Title,Description,Writer,Artist,Colorist,Cover Artist,Publication Date,Format,Price,UPC,Editor,Letterer,Penciller,Inker

# titulo,autor,periodicidade,data_publicacao,editora,issn

author_cols = ['Writers','Pencilers','Inkers','Cover_Artists']

df = pd.read_csv("revistas/comic_list.csv", dtype=str, sep=",")

pubdates = []
author_series, periodicidade = [], []
for _, row in df.iterrows():
    periodicidade.append(random.choice(['Diário', 'Semanal', 'Quinzenal', 'Mensal']))

    year = str(row['Year']).strip()
    year = year if year.isdecimal() else 2002
    pubdates.append(f"{year}-01-01")

df = df[['Series']].rename(columns={'Series': 'titulo'})
df['titulo'] = df['titulo'].str.replace(" [m]", " - ").str.replace(" []", " - ").str.strip()
df['editora'] = pd.Series([random.choice(COMIC_PUBLISHER) for _ in range(df.shape[0])])
df['autores'] = 'Cartoon Man'
df['periodicidade'] = pd.Series(periodicidade)
df['ISSN'] = pd.Series([faker_en.isbn13() for _ in range(df.shape[0])])
df['data_publicacao'] = pd.Series(pubdates)

df = df[['titulo', 'autores', 'periodicidade', 'data_publicacao', 'editora', 'ISSN']]
df.to_csv("revistas/treated/comic_list.csv", header=True, index=False)
df.head()

,titulo,autores,periodicidade,data_publicacao,editora,ISSN
0,Marvel Comics -,Cartoon Man,Semanal,1939-01-01,Tokyopop (USA division),978-0-01-849661-3
1,Marvel Mystery Comics -,Cartoon Man,Quinzenal,1939-01-01,First Second Books,978-1-09-872580-8
2,Daring Mystery Comics -,Cartoon Man,Diário,1940-01-01,BOOM! Studios,978-1-261-25499-9
3,The Human Torch -,Cartoon Man,Mensal,1940-01-01,Seven Seas Entertainment,978-0-7586-9079-1
4,Mystic Comics -,Cartoon Man,Mensal,1940-01-01,Ahoy Comics,978-1-260-48064-1


##  Artigos

In [16]:

os.remove("artigos/treated/arXiv_scientific_dataset.csv")
for idx, df in enumerate(pd.read_csv("artigos/arXiv_scientific dataset.csv", iterator=True, chunksize=CHUNKSIZE)):
    autores, data = [], []
    for _, row in df.iterrows():
        tokens = row['published_date'].split('/')
        month = tokens[0] if len(tokens[0].strip()) == 2 else '0' + tokens[0]
        day = tokens[1] if len(tokens[1].strip()) == 2 else '0' + tokens[1]
        year = '19' + tokens[2] if int(tokens[2]) > 30 else '20' + tokens[2]

        date = f'{year}-{month}-{day}'
        data.append(date)

        autores.append(' | '.join(eval(row['authors'])))
        
    df = df[['title']].rename(columns={'title': 'titulo'})
    df['DOI'] = pd.Series([faker_en.doi() for _ in range(df.shape[0])])
    df['titulo'] = df['titulo'].str.replace("\n", '')
    df['data_publicacao'] = pd.Series(data, dtype=str)
    df['autores'] = pd.Series(autores)
    df['publicadora'] = 'ArXiv'

    df = df[['titulo', 'DOI', 'publicadora', 'data_publicacao', 'autores']]
    df.to_csv("artigos/treated/arXiv_scientific_dataset.csv", mode='a', header=True if idx == 0 else False, index=False)

# titulo, DOI, publicadora, data_publicacao
df = pd.read_csv("artigos/treated/arXiv_scientific_dataset.csv", dtype=str, sep=",", nrows=1000)

df.head(1000)


,titulo,DOI,publicadora,data_publicacao,autores
0,Dynamic Backtracking,10.70048022/n5z7t43,ArXiv,1993-08-01,M. L. Ginsberg
1,A Market-Oriented Programming Environment and ...,10.95565387/f1g1q93,ArXiv,1993-08-01,M. P. Wellman
2,An Empirical Analysis of Search in GSAT,10.45390810/r7f6o68,ArXiv,1993-09-01,I. P. Gent | T. Walsh
3,The Difficulties of Learning Logic Programs wi...,10.48415449/r4p5o50,ArXiv,1993-11-01,F. Bergadano | D. Gunetti | U. Trinchero
4,Software Agents: Completing Patterns and Const...,10.33232161/u6t4y06,ArXiv,1993-11-01,J. C. Schlimmer | L. A. Hermens
...,...,...,...,...,...
995,Optimal Fuzzy Model Construction with Statisti...,10.51666039/r3b6m78,ArXiv,2012-01-10,Md. Amjad Hossain | Pintu Chandra Shill | Bish...
996,"Ultrametric Model of Mind, I: Review",10.23426428/l4f0b90,ArXiv,2012-01-13,Fionn Murtagh
997,Tacit knowledge mining algorithm based on ling...,10.22907969/w2u3v38,ArXiv,2012-01-15,Li Yang | Yuhui Wang
998,"Evaluation of a Simple, Scalable, Parallel Bes...",10.56113588/j5m2y58,ArXiv,2012-01-16,Akihiro Kishimoto | Alex Fukunaga | Adi Botea


In [17]:


df = pd.read_csv("artigos/medium_data.csv", dtype=str, sep=",")

df = df[['title','date']].rename(columns={'title': 'titulo', 'date': 'data_publicacao'})

df['publicadora'] = 'Medium'
df['DOI'] = pd.Series([faker_en.doi() for _ in range(df.shape[0])])
df['autores'] = pd.Series([faker_en.name() for _ in range(df.shape[0])])


df = df[['titulo', 'DOI', 'publicadora', 'data_publicacao', 'autores']]
df.to_csv("artigos/treated/medium_data.csv", header=True, index=False)
df.head()

,titulo,DOI,publicadora,data_publicacao,autores
0,A Beginner’s Guide to Word Embedding with Gens...,10.4345000/l3j2m85,Medium,2019-05-30,Chandani Tak
1,Hands-on Graph Neural Networks with PyTorch & ...,10.62661319/s1n4p52,Medium,2019-05-30,Bhanumati Mane
2,How to Use ggplot2 in Python,10.54733905/p6n7v38,Medium,2019-05-30,Yagnesh Kale
3,Databricks: How to Save Files in CSV on Your L...,10.28908090/q0j8p96,Medium,2019-05-30,Vedhika Tak
4,A Step-by-Step Implementation of Gradient Desc...,10.8234778/s2u7p00,Medium,2019-05-30,Jackson Nath


## DVDs

In [18]:
df = pd.read_csv("dvds/Anime.csv", dtype=str, sep=",")


df = df[['English','Duration_Minutes','Studios','Start_Aired',]].rename(columns={'English': 'titulo', 'Duration_Minutes': 'duracao', 'Studios': 'distribuidora', 'Start_Aired': 'data_lancamento'})
df['ISAN'] = pd.Series([faker_en.isbn13() for _ in range(df.shape[0])])
df['data_lancamento'] = pd.to_datetime(df['data_lancamento'], format='%b %d, %Y', errors='coerce').dt.strftime('%Y-%m-%d')


df = df[['titulo','ISAN','duracao','distribuidora','data_lancamento']]
df.to_csv("dvds/treated/Anime.csv", header=True, index=False)
df.head()

,titulo,ISAN,duracao,distribuidora,data_lancamento
0,Attack on Titan,978-1-291-41560-5,24,Wit Studio,2013-04-07
1,Death Note,978-1-893816-45-9,23,Madhouse,2006-10-04
2,Fullmetal Alchemist Brotherhood,978-0-657-89210-4,24,Bones,2009-04-05
3,One Punch Man,978-1-356-16585-8,24,Madhouse,2015-10-05
4,Sword Art Online,978-0-8178-0094-9,23,A-1 Pictures,2012-07-08


In [19]:
df = pd.read_csv("dvds/movies.csv", dtype=str, sep=",")


df = df[['title','runtime','production_companies','release_date',]].rename(columns={'title': 'titulo', 'runtime': 'duracao', 'production_companies': 'distribuidora', 'release_date': 'data_lancamento'})
df['ISAN'] = pd.Series([faker_en.isbn13() for _ in range(df.shape[0])])
df['duracao'] = df['duracao'].fillna(110).astype(float).astype(int)


df = df[['titulo','ISAN','duracao','distribuidora','data_lancamento']]
df.to_csv("dvds/treated/movies.csv", header=True, index=False)
df.head()

,titulo,ISAN,duracao,distribuidora,data_lancamento
0,Meg 2: The Trench,978-0-8087-0233-7,116,Apelles Entertainment-Warner Bros. Pictures-di...,2023-08-02
1,The Pope's Exorcist,978-0-508-01993-3,103,Screen Gems-2.0 Entertainment-Jesus & Mary-Wor...,2023-04-05
2,Deadpool & Wolverine,978-1-77639-876-8,128,Marvel Studios-Maximum Effort-21 Laps Entertai...,2024-07-24
3,Transformers: Rise of the Beasts,978-1-5257-9907-5,127,Skydance-Paramount-di Bonaventura Pictures-Bay...,2023-06-06
4,Dune: Part Two,978-0-02-943084-2,167,Legendary Pictures,2024-02-27


## Tratar as bases 

In [20]:
LIVRO_PATH = "livros/treated/"
ARTIGO_PATH = "artigos/treated/"
DVD_PATH = "dvds/treated/"
REVISTA_PATH = "revistas/treated/"

In [21]:
del df 

livros = pd.concat([pd.read_csv(path_join(LIVRO_PATH, file), dtype=str, sep=",") for file in os.listdir(LIVRO_PATH)], ignore_index=True)
artigos = pd.concat([pd.read_csv(path_join(ARTIGO_PATH, file), dtype=str, sep=",") for file in os.listdir(ARTIGO_PATH)], ignore_index=True)
dvds = pd.concat([pd.read_csv(path_join(DVD_PATH, file), dtype=str, sep=",") for file in os.listdir(DVD_PATH)], ignore_index=True)
revistas = pd.concat([pd.read_csv(path_join(REVISTA_PATH, file), dtype=str, sep=",") for file in os.listdir(REVISTA_PATH)], ignore_index=True)


## Importar para o Postgres

1. Bibliotecas (OK)
2. Usuarios (OK)
3. Autores
4. Livros
5. Revistas
6. DVD's 
7. Artigos
8. Autorias
9. Midia
10. Emprestimos
11. Penalização

In [38]:
import psycopg2
from psycopg2.extras import execute_values

# Conexão com o banco de dados PostgreSQL
conn = psycopg2.connect(
    dbname="onixlibrary",
    user="super_user",
    password="carimboatrasado",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# Função para inserir dados no banco
def insert_data(table_name, data, columns):
    query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES %s"
    execute_values(cursor, query, data)
    conn.commit()

def generate_user():
    user = {}
    user['name'] = faker_br.name()
    user['email'] = faker_br.email()
    user['address'] = ' - '.join(faker_br.address().split('\n'))
    user['phone_number'] = faker_br.phone_number()
    return user

def generate_user_list():
    user = []
    user.append(faker_br.name())
    user.append(faker_br.email())
    user.append(' - '.join(faker_br.address().split('\n')))
    user.append(faker_br.phone_number())
    return user



In [43]:
df = pd.read_sql("SELECT * FROM usuario", conn)

df.head(10)


/tmp/ipykernel_7363/2199291454.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM usuario", conn)


,id_usuario,nome,endereco,email,telefone
0,12400392,Breno da Silva Nogueira,"Rua Andaçu, 6 - Vila Paiva - São Paulo - São P...",breno.s.nogueira@gmail.com,11 968479148
1,12400394,Yan Marques,"Esplanada Samuel Oliveira, 10 - Minas Brasil -...",vsampaio@example.org,+55 (031) 3525 6300
2,12400395,Sr. Pedro Miguel Cunha,"Viela Barros, 78 - Vila Jardim Montanhes - 992...",fernandesemilly@example.org,31 9706-0055
3,12400396,Sr. Anthony Gabriel Moura,"Praia Aragão, 30 - Nova America - 13009176 da ...",aylla23@example.com,+55 71 1073 6200
4,12400397,Allana Viana,"Estação de Cavalcanti, 5 - São Vicente - 80383...",camposoliver@example.org,11 5328-5232
5,12400398,Mariah Cavalcante,"Trecho Siqueira, 8 - São Geraldo - 80493-229 A...",ofarias@example.org,51 0221 0575
6,12400399,Jade Cunha,"Praia Guerra, 86 - Coração De Jesus - 48744-89...",antonyandrade@example.com,+55 41 1956-7396
7,12400400,Luiz Miguel Casa Grande,"Fazenda de Nogueira, 3 - Senhor Dos Passos - 4...",da-rosasamuel@example.com,+55 (061) 5049-5444
8,12400401,Alexia Araújo,"Esplanada Montenegro, 56 - Vila Bandeirantes -...",maria-clarapimenta@example.com,+55 (021) 5870 5998
9,12400402,Mariah Brito,Largo Yago Campos - Etelvina Carneiro - 41151-...,pachecosofia@example.net,(041) 1055-6084


In [39]:

# # Crie uma lista de bibliotecas fictícias
# fake_libraries = [
#     "Biblioteca Onix Jaçanã",
#     "Biblioteca Onix Tatuapé",
#     "Biblioteca Onix Vila Prudente",
#     "Biblioteca Onix Paulista",
#     "Biblioteca Onix Santana",
#     "Biblioteca Onix Santo André",
#     "Biblioteca Onix São Bernardo do Campo",
#     "Biblioteca Onix São Caetano do Sul",
#     "Biblioteca Onix São José dos Campos",
#     "Biblioteca Onix Higienópolis",
#     "Biblioteca Onix Vila Madalena",
# ]
# fake_library_addresses = [
#     "Rua das Palmeiras, 123 – Jaçanã, São Paulo – SP, 02260-000",
#     "Av. Álvaro Ramos, 456 – Tatuapé, São Paulo – SP, 03310-000",
#     "Rua José Zappi, 789 – Vila Prudente, São Paulo – SP, 03138-000",
#     "Av. Paulista, 1001 – Bela Vista, São Paulo – SP, 01311-100",
#     "Rua Voluntários da Pátria, 321 – Santana, São Paulo – SP, 02010-000",
#     "Rua General Glicério, 159 – Centro, Santo André – SP, 09015-330",
#     "Av. Faria Lima, 987 – Centro, São Bernardo do Campo – SP, 09710-000",
#     "Rua Alegre, 202 – Santa Paula, São Caetano do Sul – SP, 09560-300",
#     "Av. Adhemar de Barros, 1550 – Jardim São Dimas, São José dos Campos – SP, 12245-010",
#     "Rua Itacolomi, 415 – Higienópolis, São Paulo – SP, 01239-000",
#     "Rua Harmonia, 678 – Vila Madalena, São Paulo – SP, 05435-001",
# ]
# insert_data("biblioteca", 
#             [[bib, add] for bib, add in zip(fake_libraries, fake_library_addresses)], 
#             ['nome', 'endereco'])


In [46]:
df = pd.read_csv("livros/authors.csv", dtype=str, sep=",")

df['author_id'].astype(int).max()

654021

In [ ]:


pubdate = faker_br.date_between(start_date='-30y', end_date='today').strftime('%Y-%m-%d')

insert_data("usuario", 
            [generate_user_list() for _ in range(100_000)], 
            ['nome', 'email', 'endereco', 'telefone'])



dados_convertidos = [
    (titulo, isan, duracao, distribuidora, dt.strptime(data, "%Y-%m-%d").date())
    for titulo, isan, duracao, distribuidora, data in dados
]
    

In [41]:
# # Exemplo de inserção para livros
# livros_data = [
#     (row['titulo'], row['DOI'], row['publicadora'], row['data_publicacao'], row['autores'])
#     for row in pd.read_csv("artigos/treated/arXiv_scientific_dataset.csv", dtype=str).to_dict('records')
# ]
# insert_data("livros", livros_data, ['titulo', 'DOI', 'publicadora', 'data_publicacao', 'autores'])

# # Exemplo de inserção para revistas
# revistas_data = [
#     (row['titulo'], row['autores'], row['periodicidade'], row['data_publicacao'], row['editora'], row['ISSN'])
#     for row in pd.read_csv("revistas/treated/goodreads_books_comics_graphic.csv", dtype=str).to_dict('records')
# ]
# insert_data("revistas", revistas_data, ['titulo', 'autores', 'periodicidade', 'data_publicacao', 'editora', 'ISSN'])

# # Exemplo de inserção para DVDs
# dvds_data = [
#     (row['titulo'], row['ISAN'], row['duracao'], row['distribuidora'], row['data_lancamento'])
#     for row in pd.read_csv("dvds/treated/Anime.csv", dtype=str).to_dict('records')
# ]
# insert_data("dvds", dvds_data, ['titulo', 'ISAN', 'duracao', 'distribuidora', 'data_lancamento'])

# # Fechar conexão
# cursor.close()
# conn.close()